In [1]:
import django
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'RiskconcileData.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from RiskconcileData.db.models import Regulation, RegulationRelation
import pandas as pd
from typing import Union
from py2neo import Graph


In [2]:
host = 'bolt://localhost:7687'
username = ''
password = ''

In [3]:
auth = (username, password)
graph = Graph(host, auth=auth)

<h1> Nodes and attributes </h1>

In [4]:
def split_data(
        data: pd.DataFrame,
        data_present: pd.DataFrame,
        join_variable: Union[str, list]
        ) -> tuple:
    """ Splits the data on in three different datasets:
       1. To create: contains the nodes that need to be created
       2. To delete: contains the nodes that need to be deleted
       3. To update: contains the nodes that need to be updated

    Args:
        data (pd.DataFrame): the newly loaded data that needs to be 
            uploaded to Neo4J
        data_present (pd.DataFrame): the data currently in Neo4j
        join_variable (str): the variable on which the datasets are 
            joined. This usually corresponds to the key of the data.

    Returns:
        tuple: a tuple containing the to_create, to_delete, and to_update
            dataframes.

    Example usage
    -------------
    to_create, to_delete, to_update = split_data(
        data=relations,
        data_present=existing_relations,
        join_variable=["id", "industry"]
    )
    """
    assert_message = "Please provide data as a dataframe, " \
            + "if you provided a single " \
            + "column dataframe (i.e. a pandas series), be sure to use " \
            + "the Series.to_frame() method."
    assert type(data) is pd.DataFrame, assert_message
    assert type(data_present) is pd.DataFrame, assert_message

    if data_present.empty:
        to_create = data.copy()
        to_delete = pd.DataFrame()
        to_update = pd.DataFrame()
    else:
        if type(join_variable) is str:
            assert join_variable in data.columns, \
                f"'{join_variable}' not in data."
            assert join_variable in data_present.columns, \
                f"'{join_variable}' not in present data."
        elif type(join_variable) is list:
            for variable in join_variable:
                assert variable in data.columns, \
                f"'{variable}' not in data."
                assert variable in data_present.columns, \
                    f"'{variable}' not in present data."
        temp = pd.merge(data, 
                data_present[join_variable], # Avoids the creation of variable_x, variable_y type of columns
                on=join_variable,
                how="outer", 
                indicator=True)
        to_create = temp[temp._merge == "left_only"].copy()
        to_delete = temp[temp._merge == "right_only"].copy()
        to_update = temp[temp._merge == "both"].copy()
    return to_create, to_delete, to_update


def create_nodes(
        data: pd.DataFrame,
        name: str,
        attributes: list,
        attribute_names: list,
        graph: Graph) -> str:
    """ Create new nodes with a certain name and attributes based on 
    data that can be found in a pandas dataframe.

    Args:
        data (pd.DataFrame): the dataframe containing all node information
        name (str): the name of the node that needs to be created (Neo4J label)
        attributes (list): the attributes as can be found in the database
            columns
        attribute_names (list): the attributes as you would like them
            to appear in the neo4j database. Same order as attributes.
        graph (Graph): the neo4j database to which you would like to
            upload the data,

    Returns:
        str: simple success message
    """
    assert all(elem in data.columns for elem in attributes), \
        "Ensure that all attributes are columns in the dataframe"
    assert len(attributes) == len(attribute_names), \
        "Ensure that the attributes and attribute names have the same length"
    attribute_string='{'
    for i in range(len(attributes)):
        if "_date" not in attribute_names[i]:
            attribute_string += f'{attribute_names[i]}: row.{attributes[i]}'
        else:
            attribute_string += f'{attribute_names[i]}: date(row.{attributes[i]})'
        if i < len(attributes) - 1:
            attribute_string += ','
    attribute_string += '})'
    upload_query = f'''
        UNWIND $rows AS row
        MERGE (n:{name}
        {attribute_string}
    '''
    graph.run(upload_query, parameters = {'rows': pd.DataFrame(data).to_dict('records')})
    return 'SUCCESS'

def delete_nodes(data: pd.DataFrame,
            name: str,
            attributes: list,
            attribute_names: list,
            graph: Graph) -> str:
    """ Deletes the nodes that can be found in the dataframe

    Args:
        data (pd.DataFrame): dataframe consisting of the observations
            that need to be removed.
        name (str): the name of the nodes
        attributes (list): the attributes on which nodes are matched as
            can be found in the dataframe
        attribute_names (list): the attributes on which nodes are matched
            as found in the graph database
        graph (Graph): the graph database connection

    Returns:
        str: success indicator
    """
    if not data.empty:
        query = "UNWIND $rows AS row MATCH (n:"
        query += name + '{'
        query += attributes[0] + ': row.'
        query += attribute_names[0] + '})'
        query += " detach delete n"
        graph.run(query,
             parameters = {'rows': pd.DataFrame(data).to_dict(orient='records')})
    return 'SUCCESS'


def update_nodes(data: pd.DataFrame,
                data_present: pd.DataFrame,
                name: str,
                matching_attribute: str,
                matching_attribute_name: str,
                attributes: list,
                attribute_names: list,
                graph: Graph) -> str:
    """ This function updates the nodes that are present in both the 
    dataframe and the graph database. Only those nodes that actually
    changed are updated.

    Args:
        data (pd.DataFrame): The dataframe containing the most up-to-date
        data
        data_present (pd.DataFrame): the data currently in the graph 
        database
        name (str): the name of the nodes that need to be updated
        matching_attribute (str): the attributes on which nodes are
        matched as found in the dataframe
        matching_attribute_name (str): the attributes on which nodes are
        matched as found in the graph database
        attributes (list): the attributes that need to be updated as 
        found in the data
        attribute_names (list): the attributes that need to be updated
        as found in the graph database
        graph (Graph): the graph database connection

    Returns:
        str: success indicator
    """
    if not data_present.empty:
        to_update = pd.merge(
            data_present,
            data,
            on=attributes.copy().append(matching_attribute),
            how='outer',
            indicator="ind"
        )
        changed_data = to_update[to_update.ind == "right_only"].copy()
        sets = ""
        for i in range(len(attributes)):
            sets += f"SET n.{attribute_names[i]} = row.{attributes[i]} \n"
        query = f"UNWIND $rows AS row MATCH (n:{name} "
        query += "{" + matching_attribute_name + ": row."
        query += matching_attribute + "})"
        query += sets
        graph.run(query, parameters = {'rows': changed_data.to_dict('records')})
    return "SUCCESS"


In [ ]:
from datetime import date, timedelta
today = date.today()
week_prior =  today - timedelta(weeks=1)

df_regulation = pd.DataFrame(Regulation.objects.all().values())
df_relation = pd.DataFrame(RegulationRelation.objects.all().values())

def merge_Reg_Rel_tables(reg_table, rel_table):

    reg_table = reg_table.rename(columns={"doc_code": "source_id"})
    # Merge using "source" column
    merged_table =pd.merge(rel_table,reg_table, on = "source_id", how = 'left')
    merged_table=merged_table.rename(columns={"name": "name_src", "celex_code": "celex_code_src","title": "title_src", "publication_date": "publication_date_src", "doc_type":"doc_type_src","doc_type_code": "doc_type_src", "author": "author_src", "languages":"languages_src", "url": "url_src"})

    # Merge using "target" column
    reg_table = reg_table.rename(columns={"source_id": "target_id"})
    merged_table = merged_table.merge(reg_table, how = 'left', on = "target_id")
    merged_table=merged_table.rename(columns={"name": "name_trg", "celex_code": "celex_code_trg","title": "title_trg", "publication_date": "publication_date_trg", "doc_type":"doc_type_trg","doc_type_code": "doc_type_trg", "author": "author_trg", "languages":"languages_trg", "url": "url_trg"})
    merged_table = merged_table.fillna(value = "Not found")
    merged_table = merged_table.drop_duplicates()

    return merged_table

merge_Reg_Rel_tables(df_regulation,df_relation )



df_regulation_merged  = merge_Reg_Rel_tables(df_regulation,df_relation )

def create_new_nodes(data: pd.DataFrame):
    data["publication_date"] = data["publication_date"].astype("string").str.extract('(\d{2}\/\d{2}\/\d{4})').fillna('Not found')
  
    return data

def create_author_nodes(data: pd.DataFrame):
    df = data[["doc_code", "author"]]
    df["author"] = df["author"].astype(str).str.split(", ").replace("'", "")
    df = df.explode("author")
    return df

def create_lang_nodes(data: pd.DataFrame):
    df = data[["doc_code", "languages"]]
    df["languages"] = df["languages"].astype(str).str.split(", ").replace("'", "")
    df = df.explode("languages")
    return df

def create_class_nodes(data: pd.DataFrame):
    df = data[["doc_code", "classification"]]
    df["classification"] = df["classification"].replace("'", "", regex = True).replace("\[", "", regex = True).replace("\]", "", regex = True)
    df["classification"] = df["classification"].astype(str).str.split(", ").replace("'", "")

    df = df.explode("classification")
    return df

In [9]:
node_name: str = 'regulation'

types = pd.Series(df_regulation.doc_code.unique(), name="doc_code")
types_present = graph.run(f'match (s:{node_name}) return s.doc_code as doc_code').to_data_frame()

In [10]:
to_create, to_delete, to_update = split_data(
    data=types.to_frame(),
    data_present=types_present,
    join_variable="doc_code"
)

In [ ]:
create_nodes(df_regulation_merged, "regulation", ["source_id", "celex_code_src","title_src", "quicksearch_url_x", "date_effect_x", "date_sig_x", "date_deadline_x"], ["doc_code","celex_code", "title",  "quicksearch_url", "date_effect", "date_signature", "deadlines"], graph)
create_nodes(df_regulation_merged, "regulation", ["target_id", "celex_code_trg","title_trg", "quicksearch_url_y", "date_effect_y", "date_sig_y", "date_deadline_y"], [ "doc_code","celex_code","title",  "quicksearch_url", "date_effect", "date_signature", "deadlines"], graph)
create_nodes(create_new_nodes(df_regulation), "date", ["publication_date"], ["date"], graph)
create_nodes(create_author_nodes(df_regulation), "author", ["author"], ["author"], graph)
create_nodes(create_lang_nodes(df_regulation), "languages", ["languages"], ["languages"], graph)
create_nodes(create_class_nodes(df_regulation), "classification", ["classification"], ["classification"], graph)
create_nodes(df_regulation, "type", ["doc_type"], ["doc_type"], graph)
create_nodes(df_regulation, "name", ["name"], ["name"], graph)

<h1> Relations </h1>

In [24]:
def create_relations(data: pd.DataFrame,
                    source_node_name: str,
                    dest_node_name: str,
                    source_node_attribute: str,
                    dest_node_attribute: str,
                    source_node_attribute_name: str,
                    dest_node_attribute_name: str,
                    relation_name: str,
                    graph: Graph,
                    relation_attributes: list=None,
                    relation_attribute_names: list=None) -> str:
    """ This function creates relationships between nodes based on
    data in a dataframe.

    Args:
        data (pd.DataFrame): the data that needs to be created
        source_node_name (str): the name of the source node
        dest_node_name (str): the name of the destination node
        source_node_attribute (str): the attributes of the source node
            used for matching as they appear in the data
        dest_node_attribute (str): the attributes of the target node
            used for matching as they appear in the data
        source_node_attribute_name (str): the attributes of the source node
            used for matching as they appear in the graph database
        dest_node_attribute_name (str): the attributes of the target node
            used for matching as they appear in the graph database
        relation_name (str): the name of the relation
        graph (Graph): graph database
        relation_attributes (list, optional): the attributes for the 
            relation as they appead in the data. Defaults to None.
        relation_attribute_names (list, optional): the attributes for the 
            relation as they appead in the graph database. Defaults to None.

    Returns:
        str: success indicator
    """
    attribute_string = ''
    if relation_attributes is not None \
            and relation_attribute_names is not None:
        assert len(relation_attributes) == len(relation_attribute_names), \
            "Ensure that the relation attributes and relation attribute names have the same length"
        attribute_string='{'
        for i in range(len(relation_attributes)):
            # Dates need to be explicitly created as dates in neo4j. Since in 
            # our data, a date is always found in a column that contains the 
            # string _date in its name, we can handle them using a condition
            if "_date" not in relation_attribute_names[i]:
                attribute_string += f'{relation_attribute_names[i]}: row.{relation_attributes[i]}'
            else:
                attribute_string += f'{relation_attribute_names[i]}: date(row.{relation_attributes[i]})'
            if i < len(relation_attributes) - 1:
                attribute_string += ','
        attribute_string += '}'
    query = "UNWIND $rows AS row \n"
    query += f"MATCH (n1:{source_node_name} " + '{'
    query += source_node_attribute_name + f': row.{source_node_attribute}' + '}), '
    query += f"(n2:{dest_node_name} " + '{'
    query += dest_node_attribute_name + f': row.{dest_node_attribute}' + '}) \n'
    query += f"CREATE (n1) -[rel:{relation_name} {attribute_string}]-> (n2) \n RETURN count(*) as total "
    graph.run(query, parameters = {'rows': data.to_dict('records')})
    return "SUCCESS"


def delete_relations(data: pd.DataFrame,
                    source_node_name: str,
                    dest_node_name: str,
                    source_node_attribute: str,
                    dest_node_attribute: str,
                    source_node_attribute_name: str,
                    dest_node_attribute_name: str,
                    relation_name: str,
                    graph: Graph):
    if not data.empty:
        data = data[[source_node_attribute, dest_node_attribute]]
        query = "UNWIND $rows AS row \n"
        query += f"MATCH (n1:{source_node_name} " + "{" + source_node_attribute_name + f": row.{source_node_attribute}" + "}) "
        query += f"-[rel:{relation_name}]-> (n2:{dest_node_name}" + "{"
        query += dest_node_attribute_name + ": row." + dest_node_attribute + "}) \n"
        query += "DELETE rel"
        graph.run(query,
                 parameters = {'rows': data.to_dict(orient='records')})
    return 'SUCCESS'


def update_relations(relations: pd.DataFrame,
                relations_present: pd.DataFrame,
                name: str,
                source_name: str,
                target_name: str,
                matching_attribute_source: str,
                matching_attribute_source_name: str,
                matching_attribute_target: str,
                matching_attribute_target_name: str,
                attributes: list,
                attribute_names: list,
                graph: Graph):
    if not relations_present.empty:
        to_update = pd.merge(
            relations_present,
            relations,
            on=attributes.copy() + [matching_attribute_source_name, matching_attribute_target_name],
            how='outer',
            indicator="ind"
        )
        changed_data = to_update[to_update.ind == "right_only"].copy()
        sets = ""
        for i in range(len(attributes)):
            # Dates need to be explicitly created as dates in neo4j. Since in 
            # our data, a date is always found in a column that contains the 
            # string _date in its name, we can handle them using a condition
            if "_date" not in attribute_names[i]:
                sets += f"SET r.{attribute_names[i]} = row.{attributes[i]} \n"
            else:
                sets += f"SET r.{attribute_names[i]} = date(row.{attributes[i]}) \n"
        query = f"UNWIND $rows AS row MATCH (e1:{source_name} \n"
        query += "{" + matching_attribute_source_name + ": row."
        query += matching_attribute_source + "}) "
        query += f"-[r:{name}]-> (e2:{target_name} "
        query += "{" + matching_attribute_target_name + ": row."
        query += matching_attribute_target + "}) \n"
        query += sets
        graph.run(query, parameters = {'rows': changed_data.to_dict('records')})
    return "SUCCESS"


def delete_relations_id(data: pd.DataFrame,
                        name: str,
                        id_column: str,
                        id_attribute_name: str,
                        graph: Graph):
    query = f"""
        UNWIND $rows AS row
        MATCH (e1) -[r:{name}]- (e2)
        WHERE r.{id_attribute_name} = row.{id_column}
        DELETE r
    """
    graph.run(query, parameters = {'rows': data.to_dict(orient='records')})
    return 'SUCCESS'


def update_relations_id(relations: pd.DataFrame,
                        relations_present: pd.DataFrame,
                        name: str,
                        id_column: str,
                        id_attribute_name: str,
                        attributes: list,
                        attribute_names: list,
                        graph: Graph):
    if not relations_present.empty:
        to_update = pd.merge(
            relations_present,
            relations,
            on=attributes.copy() + [id_column],
            how='outer',
            indicator="ind"
        )
        changed_data = to_update[to_update.ind == "right_only"].copy()
        sets = ""
        for i in range(len(attributes)):
            # Dates need to be explicitly created as dates in neo4j. Since in 
            # our data, a date is always found in a column that contains the 
            # string _date in its name, we can handle them using a condition
            if "_date" not in attribute_names[i]:
                sets += f"SET r.{attribute_names[i]} = row.{attributes[i]} \n"
            else:
                sets += f"SET r.{attribute_names[i]} = date(row.{attributes[i]}) \n"
        query = f"""
            UNWIND $rows AS row
            MATCH (e1) -[r:{name}]- (e2)
            WHERE r.{id_attribute_name} = row.{id_column}
        """
        query += sets
        graph.run(query, parameters = {'rows': changed_data.to_dict('records')})
    return "SUCCESS"

In [27]:
df_amending = df_relation[df_relation["relation_type"]== "amending"]
df_supplementing = df_relation[df_relation["relation_type"]== "supplementing"]
df_referencing = df_relation[df_relation["relation_type"]== "references"]

create_relations(df_amending, "regulation", "regulation","source_id", "target_id", "doc_code", "doc_code", "AMENDING", graph)
create_relations(df_supplementing, "regulation", "regulation","source_id", "target_id","doc_code", "doc_code","SUPPLEMENTING", graph)
create_relations(df_referencing, "regulation", "regulation", "source_id", "target_id","doc_code", "doc_code","REFERENCES", graph)
create_relations(df_regulation,"regulation", "date", "doc_code", "publication_date","doc_code", "date", "PUBLISHED", graph )
create_relations(df_regulation,"regulation", "author", "doc_code", "author","doc_code", "author", "AUTHOR", graph )
create_relations(df_regulation,"regulation", "languages", "doc_code", "languages","doc_code", "languages", "language", graph )
create_relations(df_regulation,"regulation", "classification", "doc_code", "classification","doc_code", "classfification", "CATEGORY", graph )
create_relations(df_regulation,"regulation", "author", "doc_code", "author","doc_code", "author", "AUTHOR", graph )
create_relations(df_regulation,"regulation", "type", "doc_code", "doc_type","doc_code", "doc_type", "TYPE", graph )
create_relations(df_regulation,"regulation", "name", "doc_code", "name","doc_code", "name", "NAME", graph )


'SUCCESS'